# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f33cf192e80>
├── 'a' --> tensor([[-1.7619, -0.3400, -0.7624],
│                   [ 0.5088,  1.0374, -0.0132]])
└── 'x' --> <FastTreeValue 0x7f33cf1175e0>
    └── 'c' --> tensor([[ 0.9887,  0.9275,  1.0844,  1.7493],
                        [ 0.2160, -0.1190, -0.5881,  1.6509],
                        [-1.3378, -0.1905, -0.3306, -0.6152]])

In [4]:
t.a

tensor([[-1.7619, -0.3400, -0.7624],
        [ 0.5088,  1.0374, -0.0132]])

In [5]:
%timeit t.a

74.9 ns ± 0.76 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f33cf192e80>
├── 'a' --> tensor([[-2.3170, -1.5246,  0.8002],
│                   [-0.9990,  0.1658,  1.1647]])
└── 'x' --> <FastTreeValue 0x7f33cf1175e0>
    └── 'c' --> tensor([[ 0.9887,  0.9275,  1.0844,  1.7493],
                        [ 0.2160, -0.1190, -0.5881,  1.6509],
                        [-1.3378, -0.1905, -0.3306, -0.6152]])

In [7]:
%timeit t.a = new_value

82.8 ns ± 1.07 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.7619, -0.3400, -0.7624],
               [ 0.5088,  1.0374, -0.0132]]),
    x: Batch(
           c: tensor([[ 0.9887,  0.9275,  1.0844,  1.7493],
                      [ 0.2160, -0.1190, -0.5881,  1.6509],
                      [-1.3378, -0.1905, -0.3306, -0.6152]]),
       ),
)

In [10]:
b.a

tensor([[-1.7619, -0.3400, -0.7624],
        [ 0.5088,  1.0374, -0.0132]])

In [11]:
%timeit b.a

73.7 ns ± 0.657 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.0152,  0.9930, -0.1962],
               [-1.3518,  0.1361, -1.1091]]),
    x: Batch(
           c: tensor([[ 0.9887,  0.9275,  1.0844,  1.7493],
                      [ 0.2160, -0.1190, -0.5881,  1.6509],
                      [-1.3378, -0.1905, -0.3306, -0.6152]]),
       ),
)

In [13]:
%timeit b.a = new_value

668 ns ± 9.08 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.09 µs ± 15.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

15.7 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

319 µs ± 28.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

308 µs ± 28.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f32ff311400>
├── 'a' --> tensor([[[-1.7619, -0.3400, -0.7624],
│                    [ 0.5088,  1.0374, -0.0132]],
│           
│                   [[-1.7619, -0.3400, -0.7624],
│                    [ 0.5088,  1.0374, -0.0132]],
│           
│                   [[-1.7619, -0.3400, -0.7624],
│                    [ 0.5088,  1.0374, -0.0132]],
│           
│                   [[-1.7619, -0.3400, -0.7624],
│                    [ 0.5088,  1.0374, -0.0132]],
│           
│                   [[-1.7619, -0.3400, -0.7624],
│                    [ 0.5088,  1.0374, -0.0132]],
│           
│                   [[-1.7619, -0.3400, -0.7624],
│                    [ 0.5088,  1.0374, -0.0132]],
│           
│                   [[-1.7619, -0.3400, -0.7624],
│                    [ 0.5088,  1.0374, -0.0132]],
│           
│                   [[-1.7619, -0.3400, -0.7624],
│                    [ 0.5088,  1.0374, -0.0132]]])
└── 'x' --> <FastTreeValue 0x7f32ff3114c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

51.1 µs ± 568 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f32ff328ca0>
├── 'a' --> tensor([[-1.7619, -0.3400, -0.7624],
│                   [ 0.5088,  1.0374, -0.0132],
│                   [-1.7619, -0.3400, -0.7624],
│                   [ 0.5088,  1.0374, -0.0132],
│                   [-1.7619, -0.3400, -0.7624],
│                   [ 0.5088,  1.0374, -0.0132],
│                   [-1.7619, -0.3400, -0.7624],
│                   [ 0.5088,  1.0374, -0.0132],
│                   [-1.7619, -0.3400, -0.7624],
│                   [ 0.5088,  1.0374, -0.0132],
│                   [-1.7619, -0.3400, -0.7624],
│                   [ 0.5088,  1.0374, -0.0132],
│                   [-1.7619, -0.3400, -0.7624],
│                   [ 0.5088,  1.0374, -0.0132],
│                   [-1.7619, -0.3400, -0.7624],
│                   [ 0.5088,  1.0374, -0.0132]])
└── 'x' --> <FastTreeValue 0x7f32ff328dc0>
    └── 'c' --> tensor([[ 0.9887,  0.9275,  1.0844,  1.7493],
                        [ 0.2160, -0.1190, -0.5881,  1.6509],
                 

In [23]:
%timeit t_cat(trees)

48.3 µs ± 3.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

100 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.7619, -0.3400, -0.7624],
                [ 0.5088,  1.0374, -0.0132]],
       
               [[-1.7619, -0.3400, -0.7624],
                [ 0.5088,  1.0374, -0.0132]],
       
               [[-1.7619, -0.3400, -0.7624],
                [ 0.5088,  1.0374, -0.0132]],
       
               [[-1.7619, -0.3400, -0.7624],
                [ 0.5088,  1.0374, -0.0132]],
       
               [[-1.7619, -0.3400, -0.7624],
                [ 0.5088,  1.0374, -0.0132]],
       
               [[-1.7619, -0.3400, -0.7624],
                [ 0.5088,  1.0374, -0.0132]],
       
               [[-1.7619, -0.3400, -0.7624],
                [ 0.5088,  1.0374, -0.0132]],
       
               [[-1.7619, -0.3400, -0.7624],
                [ 0.5088,  1.0374, -0.0132]]]),
    x: Batch(
           c: tensor([[[ 0.9887,  0.9275,  1.0844,  1.7493],
                       [ 0.2160, -0.1190, -0.5881,  1.6509],
                       [-1.3378, -0.1905, -0.3306, -0.6152]],
         

In [26]:
%timeit Batch.stack(batches)

117 µs ± 2.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.7619, -0.3400, -0.7624],
               [ 0.5088,  1.0374, -0.0132],
               [-1.7619, -0.3400, -0.7624],
               [ 0.5088,  1.0374, -0.0132],
               [-1.7619, -0.3400, -0.7624],
               [ 0.5088,  1.0374, -0.0132],
               [-1.7619, -0.3400, -0.7624],
               [ 0.5088,  1.0374, -0.0132],
               [-1.7619, -0.3400, -0.7624],
               [ 0.5088,  1.0374, -0.0132],
               [-1.7619, -0.3400, -0.7624],
               [ 0.5088,  1.0374, -0.0132],
               [-1.7619, -0.3400, -0.7624],
               [ 0.5088,  1.0374, -0.0132],
               [-1.7619, -0.3400, -0.7624],
               [ 0.5088,  1.0374, -0.0132]]),
    x: Batch(
           c: tensor([[ 0.9887,  0.9275,  1.0844,  1.7493],
                      [ 0.2160, -0.1190, -0.5881,  1.6509],
                      [-1.3378, -0.1905, -0.3306, -0.6152],
                      [ 0.9887,  0.9275,  1.0844,  1.7493],
                      [ 0.2160, -

In [28]:
%timeit Batch.cat(batches)

208 µs ± 7.57 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

684 µs ± 47.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
